In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!chmod 600 /content/kaggle.json
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p /content --force

 99% 808M/814M [00:04<00:00, 162MB/s]
100% 814M/814M [00:04<00:00, 204MB/s]


In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [ ]:
!unzip -q train.zip -d .

In [ ]:
import os
import tensorflow as tf
import shutil

print(len(os.listdir('/content/train/')))

#classify data
for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/cats/' +i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/dogs/' +i)


25000


In [ ]:
#80% of data will be used for training
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size = 64,
    subset= 'training',
    validation_split = 0.2,
    seed = 1234
)
#20% of data will be used for validation
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size = 64,
    subset= 'validation',
    validation_split = 0.2    ,
    seed = 1234
)

#make all input data are 0 ~ 1
def preprocess(i, answer):
  i = tf.cast(i/255.0, tf.float32)
  return i, answer

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape = (64,64,3) ), #image augmentation
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0,2), #prevent overfitting
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0,2),
    tf.keras.layers.Dense(1, activation='sigmoid') # output will 0~1
])

model.compile(loss ="binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])
model.fit(train_dataset, validation_data= val_dataset, epochs=5)
